In [ ]:
! pip install python-chess==0.28.3

from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
    pass
  
import tensorflow as tf

import h5py
import numpy as np
import os
import chess
import chess.pgn
from matplotlib import pyplot as plt
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
WORKING_PLACE = '/content/drive/My Drive/'

FEATURES_PATH = '/content/drive/My Drive/bitmaps_dataset.hdf5'
LABELS_PATH = '/content/drive/My Drive/labels_dataset.hdf5'

In [ ]:
# diviser l'ensemble de données en trois sous-ensembles disjoints: 
# 1. un ensemble d'entrainement  80% des données 
# 2. un ensemble de validation   10% des données 
# 3. un ensemble de test         10% des données 

validation_split = 0.1  
test_split = 0.1
with h5py.File(FEATURES_PATH, 'r') as pos_file:
    with h5py.File(LABELS_PATH, 'r') as labels_file:
        positions = pos_file.get("dataset")
        labels = labels_file.get("dataset")
      
        nb_examples = labels.shape[0]
        nb_training_examples = int(nb_examples * (1 - (validation_split + test_split)))    
        index_validation = int(nb_examples * (1 - (validation_split)))
        
        print("Nombre d'exemples : {}".format(nb_examples))

        # ensemble d'entrainement
        x_train = positions[:nb_training_examples]
        y_train = labels[:nb_training_examples]
        print("Nombre d'élements de l'ensemble d'entrainement : ", nb_training_examples)

        # ensemble de validation 
        x_validation = positions[nb_training_examples:index_validation]
        y_validation = labels[nb_training_examples:index_validation]
        print("Nombre d'éléments de l'ensemble de validation :", (index_validation - nb_training_examples ))

        # ensemble de test 
        x_test = positions[index_validation:]
        y_test = labels[index_validation:]
        print("Nombre d'éléments de l'ensemble de test :", (nb_examples - index_validation ))

        train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train))
        validation_set = tf.data.Dataset.from_tensor_slices((x_validation, y_validation))
        test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test))
        
        print(train_set)
        print(test_set)
        print(validation_set)

In [ ]:
# Batch size
BATCH_SIZE = 128

BUFFER_SIZE = 100000

train_set = train_set.shuffle(BUFFER_SIZE).batch(BATCH_SIZE,  drop_remainder=True)
validation_set = validation_set.shuffle(BUFFER_SIZE // 10).batch(BATCH_SIZE,  drop_remainder=True)
test_set = test_set.shuffle(BUFFER_SIZE // 10).batch(BATCH_SIZE,  drop_remainder=True)
train_set

In [ ]:
from tensorflow.keras import layers

In [ ]:
input_size = 773 # feature representation 

nb_units_1 = 250
nb_units_2 = 80
output_size = 1

In [ ]:
def define_model():
    model = tf.keras.Sequential()

    model = tf.keras.Sequential()
    model.add(layers.Input(shape = (input_size,)))
    model.add(layers.BatchNormalization())

    model.add(layers.Dense(nb_units_1, activation = 'relu'))
    model.add(layers.Dropout(0.2))

    model.add(layers.Dense(nb_units_2, activation = 'relu'))
    model.add(layers.Dropout(0.2))

    model.add(layers.Dense(output_size, activation = 'sigmoid'))
    return model 

In [ ]:
optimizer = tf.keras.optimizers.Adam(lr = 0.0001) # l'optimiseur Adam
model = define_model() 
model.compile(optimizer = optimizer, loss = "binary_crossentropy", metrics = ['accuracy'])
model.summary()

In [ ]:
EPOCHS = 20
history = model.fit(train_set, epochs=EPOCHS,validation_data=validation_set)
model.evaluate(test_set)

In [ ]:
loss_curve = history.history["loss"]
acc_curve = history.history["accuracy"]

loss_val_curve = history.history["val_loss"]
acc_val_curve = history.history["val_accuracy"]

plt.plot(loss_curve, label="Train")
plt.plot(loss_val_curve, label="Val")
plt.legend(loc='upper left')
plt.title("Loss")
plt.show()

plt.plot(acc_curve, label="Train")
plt.plot(acc_val_curve, label="Val")
plt.legend(loc='upper left')
plt.title("Accuracy")
plt.show()

In [ ]:
# Get the dictionary containing each metric and the loss for each epoch
history_dict = history.history
epoch_values = history.epoch
# Save it under the form of a json file
json.dump(history_dict, open(WORKING_PLACE+"modele/bitmap_training_history", 'w'))
json.dump(epoch_values, open(WORKING_PLACE+"modele/bitmap_training_epochs", 'w'))
#model.save(WORKING_PLACE+"modele/bitmap_model.h5")

In [ ]:
SMALL_SIZE = 10
MEDIUM_SIZE = 13
BIGGER_SIZE = 24

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize

In [ ]:
#load histories and epochs 
history = json.load(open(WORKING_PLACE+"modele/bitmap_training_history", 'r'))
epochs = json.load(open(WORKING_PLACE+"modele/bitmap_training_epochs", 'r'))

train_loss = history['loss']
plt.figure(figsize=(7, 6))
plt.plot(epochs, train_loss, color = 'blue', label = 'format binaire')
plt.xlabel('Nombre d\'itérations')
plt.ylabel('Fonction coût')
plt.legend()
plt.savefig(WORKING_PLACE+"modele/Fonction cout")
plt.show()


In [ ]:
train_auc = history['auc']
plt.figure(figsize=(7, 6))
plt.plot(epochs, train_auc, color = 'red', label = 'format binaire')
plt.xlabel('Nombre d\'itérations')
plt.ylabel('L\'aire sous la courbe ROC (AUC)')
plt.legend()
plt.savefig(WORKING_PLACE+"modele/Auc")
plt.show()


In [ ]:
train_auccuracy = history['accuracy']
plt.figure(figsize=(7, 6))
plt.plot(epochs, train_auccuracy, color = 'black', label = 'format binaire')
plt.xlabel('Nombre d\'itérations')
plt.ylabel('Justesse')
plt.legend()
plt.savefig(WORKING_PLACE+"modele/précision")
plt.show()
